In [1]:
!pip install emoji

In [2]:
import pandas as pd
import regex as re
import numpy as np
import matplotlib.pyplot as plt
import emoji
import plotly.express as px
from collections import defaultdict
import nltk
from collections import Counter

In [3]:
nltk.download('nps_chat')
nltk.download('punkt')

[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
myfile = open('chat.txt')
content = myfile.read()
chat = defaultdict(list)
content = content.split('\n')
for line in content:
  try:
    person = line.split(':')[2][7:]
    text = nltk.sent_tokenize(line.split(':')[3:][0])
    chat[person].extend(text)
  except:
    pass

In [5]:
def dateCheck(s):
  pattern = '\d{4}-\d{2}-\d{2}, (\d{2}|\d{1}):\d{2}:\d{2} (AM|PM|am|pm)'
  result = re.search(pattern, s)
  if result:
    return result
  return False

In [6]:
dateCheck('[2021-02-21, 10:59:44 PM] Krishna: Hi! Nice!')

<regex.Match object; span=(1, 24), match='2021-02-21, 10:59:44 PM'>

In [7]:
def getDataPoint(line):   
    splitLine = line.split(' - ') 
    dateTime = splitLine[0]
    date, time = dateTime[:dateTime.find(',')][1:],dateTime[dateTime.find(','):][1:]
    date =re.sub('\D', ' ', date).strip()
    date = date.replace(' ', '-') 
    time, author = time[:time.find(']')],time[time.find(']'):][1:]
    author, message = author[:author.find(':')],author[author.find(':'):][1:]
    return date, time.strip(), author.strip(), message.strip()

In [8]:
data = []
conversation = 'chat.txt'
with open(conversation, encoding="utf-8") as fp:
    fp.readline()
    messageBuffer = []
    parsedData = [] 
    date, time, author = None, None, None
    while True:
        line = fp.readline() 
        if not line: 
            break
        line = line.strip() 
        if dateCheck(line): 
            if len(messageBuffer) > 0: 
                parsedData.append([date, time, author, ' '.join(messageBuffer)]) 
            messageBuffer.clear()
            date, time, author, message = getDataPoint(line) 
            messageBuffer.append(message)
        else:
            messageBuffer.append(line)

In [9]:
posts = nltk.corpus.nps_chat.xml_posts()

def extract_features(post):
  features={}
  for word in nltk.word_tokenize(post):
    features['contains({})'.format(word.lower())]=True
  return features

fposts = [(extract_features(p.text),p.get('class')) for p in posts]
test_size = int(len(fposts)*0.1)
train_set,test_set = fposts[test_size:],fposts[:test_size]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [10]:
classifier.classify(extract_features('What is your score'))

'whQuestion'

In [11]:
#def emoji_used(str1):
  #return ''.join(t for t in str1 if t in emoji.UNICODE_EMOJI)

print(len(chat))
for key, val in chat.items():
  emojis = []
  for text in val:
    token = nltk.word_tokenize(text)
    for temp in token:
      if(temp in emoji.UNICODE_EMOJI_ENGLISH):
        emojis.append(temp)
  count = Counter(emojis).most_common()[:10]
  print('{}s emojis:\n{}\n'.format(key,emojis))
  print('Common: {}\n\n'.format(count))

2
Harsh Patel Computer Sciences emojis:
['👍', '👍', '👍', '👍', '👍🏻']

Common: [('👍', 4), ('👍🏻', 1)]


Krishnas emojis:
['🙄', '🤣', '✌🏻', '😅', '👍🏻']

Common: [('🙄', 1), ('🤣', 1), ('✌🏻', 1), ('😅', 1), ('👍🏻', 1)]




In [13]:
df = pd.DataFrame(parsedData, columns=['Date','Time','Author','Message'])
df.tail()
#df.to_csv('test_data.csv')

,Date,Time,Author,Message
1077,2021-05-04,11:46:14 AM,Harsh Patel Computer Science,I mean project or some research
1078,2021-05-04,11:47:18 AM,Krishna,Research itself
1079,2021-05-04,11:50:43 AM,Harsh Patel Computer Science,Ok
1080,2021-05-16,8:08:25 PM,Krishna,CS_802_Programming_Assignment.pdf • ‎1 page ‎d...
1081,2021-05-16,8:08:51 PM,Harsh Patel Computer Science,Thanks bro
